In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from activity_validator.hetus_data_processing import main
import activity_validator.hetus_data_processing.hetus_column_names as col
# load the main data
# TODO: only works when executed from project root directory
profile_type, data = main.process_hetus_2010_data()
act_data = col.get_activity_data(data)

In [ ]:
# functions for sorting the data and see how the plot changes
def sort_by_waking_up(act_data):
    """Sort by first activity that is not 'sleep'"""
    # set 'sleep' to NAN
    no_sleep = act_data[act_data != "sleep"].T
    # reset to int index (MACT1 - MACT144 is not sortable)
    no_sleep.reset_index(drop=True, inplace=True)
    # get the index of first non-NAN value for each column
    i_first  = pd.Series([no_sleep[s].first_valid_index() for s in no_sleep])
    # set an arbitrary high index for NAN values and sort the data by the indices
    i_first.fillna(200, inplace=True)
    i_sorted = i_first.argsort()
    sorted_data = act_data.iloc[i_sorted, :]
    return sorted_data

In [ ]:
sorted_data = sort_by_waking_up(act_data)
# sorted_data = sorted_data.iloc[:100,:]

In [ ]:
# encode data as ints
single_col = sorted_data.stack(sorted_data.columns.names)
codes, uniques = single_col.factorize()  # type: ignore
single_col[:] = codes
encoded = single_col.unstack()

In [ ]:
# Plot the heatmap. Further info: https://plotly.com/python/heatmaps/
heatmap = go.Heatmap(z=encoded.T, colorscale=px.colors.qualitative.Plotly)

# adapt the colorbar to use the string values
colorbar = go.heatmap.ColorBar(title='Activity Groups', tickvals=list(range(len(uniques))), ticktext=uniques)
heatmap.update(colorbar=colorbar)

fig = go.Figure(heatmap)
title = " - ".join(profile_type.to_tuple())
fig.update_layout(xaxis_title="Diary Entry", yaxis_title="Time", title=title)
# set the tooltip to show the string instead of the encoded int
fig.update_traces(hovertext=sorted_data.T)
fig.layout.yaxis.autorange="reversed"
fig.show()